In [1]:
import cirq
import numpy as np
from pathlib import Path
import stimcirq
from codes_constructions.syndrome_extraction_circuit_catalog import build_noiseless_syndrome_extraction_circuit
from codes_constructions.stabilizer_code_factory import StabilizerCodeFactory

from codes_constructions.circuit_models import (
    SeqOperationConstraints,
)
from ionq.qcircuitsim.backends.quimb.quimb_mps_simulator import QuimbMPSSimulator
from ionq.qcircuitsim.backends.cirq.cirq_qcircuit_simulator import CirqSimulatorOptions
from ionq.qcircuitsim.qcircuit_simulator.qcircuit_simulator_options import QCircuitSimulatorOptions
import sinter
import stim
import time
import sys
sys.path.append("/home/lucas.slattery/clean_repo/qec_simulations/")
from qec.benchmark_performance import mps_pipeline, executors

In [ ]:
inputs = {
    "bb_code_parameter": StabilizerCodeFactory.build_bivariate_bicycle_code_valid_parameters(),
    "constraints": SeqOperationConstraints(num_simul_CNOTs=1, num_ancillas=1),
    "lazy_indexing": [False, True],
    "chi": [1, 2, 4, 8, 16, 32],
    "num_repetitions": 1,
    "collated_result_filepath": Path("/home/lucas.slattery/clean_repo/qec_simulations/data/bb_code_mps_sim_timing_results.json"),
}
mps_pipeline.map_async(inputs,run_folder="run_folder",executor=executors(),chunksizes={"": 1})

HTML(value='\n        <style>\n            .jupyter-widgets-output-area .output_scroll {\n                heig…

AsyncMap(task=<Task pending name='Task-5' coro=<run_map_async.<locals>._run_pipeline() running at /home/lucas.slattery/micromamba/envs/system_performance/lib/python3.12/site-packages/pipefunc/map/_run.py:351>>, run_info=RunInfo(inputs={'bb_code_parameter': [(18, 4), (24, 4), (28, 6), (30, 4), (30, 8), (36, 4), (36, 8), (42, 4), (42, 6), (48, 4), (48, 8), (54, 4), (54, 8), (54, 12), (72, 12), (90, 8), (108, 8), (144, 12), (288, 12)], 'constraints': <codes_constructions.circuit_models.SeqOperationConstraints object at 0x7a543f49c2c0>, 'lazy_indexing': [False, True], 'chi': [1, 2, 4, 8, 16, 32], 'num_repetitions': 1, 'collated_result_filepath': PosixPath('/home/lucas.slattery/clean_repo/qec_simulations/data/bb_code_mps_sim_timing_results.json')}, defaults={}, all_output_names={'collated_results', 'end', 'result_dicts'}, shapes={'chi': (6,), 'lazy_indexing': (2,), 'bb_code_parameter': (19,), 'result_dicts': (19, 2, 6)}, resolved_shapes={'chi': (6,), 'lazy_indexing': (2,), 'bb_code_paramete

In [10]:
import cirq
import numpy as np
from pathlib import Path
import stimcirq
from codes_constructions.syndrome_extraction_circuit_catalog import build_noiseless_syndrome_extraction_circuit
from codes_constructions.stabilizer_code_factory import StabilizerCodeFactory

from codes_constructions.circuit_models import (
    SeqOperationConstraints,
)
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import socket
from adaptive_scheduler import SlurmExecutor
from ionq.qcircuitsim.backends.quimb.quimb_mps_simulator import QuimbMPSSimulator
from ionq.qcircuitsim.backends.cirq.cirq_qcircuit_simulator import CirqSimulatorOptions, CirqQCircuitSimulator
from ionq.qcircuitsim.qcircuit_simulator.qcircuit_simulator_options import QCircuitSimulatorOptions
import sinter
import stim
import time
from pipefunc.resources import Resources
from pipefunc import PipeFunc, Pipeline


# Build the code
bb_code_parameters =  StabilizerCodeFactory.build_bivariate_bicycle_code_valid_parameters()
constraints = SeqOperationConstraints(num_simul_CNOTs=1, num_ancillas=10)
n_data_qubits, n_logical_qubits = bb_code_parameters[10]
code = StabilizerCodeFactory.build_bivariate_bicycle_code(n_data_qubits, n_logical_qubits)
# Build the noiseless circuit
circuit = build_noiseless_syndrome_extraction_circuit(
    code,
    constraints,
)

cirq_circuit = stimcirq.stim_circuit_to_cirq_circuit(circuit)

In [ ]:
circuit

In [11]:
cirq_circuit

0: ────────────────────────@───────────────────────X───────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───────────────────────────────────────────────────────────────────X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           │                       │                                                   │                                                                                                                                                                                                                                                                                                            │                                                                                                                                                                                                                                                                                                                                                                                                         │                                                                   │
1: ────────────────────────┼───────────────────────┼───X───────────────────────────────────────────@───┼───────────────────X───────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────┼───X────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [9]:
len(cirq_circuit.all_measurement_key_names())

48

In [12]:
cirq_circuit.all_measurement_key_names()

frozenset({'0',
           '1',
           '10',
           '11',
           '12',
           '13',
           '14',
           '15',
           '16',
           '17',
           '18',
           '19',
           '2',
           '20',
           '21',
           '22',
           '23',
           '24',
           '25',
           '26',
           '27',
           '28',
           '29',
           '3',
           '30',
           '31',
           '32',
           '33',
           '34',
           '35',
           '36',
           '37',
           '38',
           '39',
           '4',
           '40',
           '41',
           '42',
           '43',
           '44',
           '45',
           '46',
           '47',
           '5',
           '6',
           '7',
           '8',
           '9'})

In [21]:
options = CirqSimulatorOptions(QCircuitSimulatorOptions(mps_max_bond_dimension = 16, mps_track_discarded_sv_norms=True, mps_lazy_indexing=True))

sim = QuimbMPSSimulator(options)

time_start = time.time()
for _ in range(1):
    result = sim.simulate(cirq_circuit)
time_stop = time.time()

In [22]:
result.final_state.estimation_stats()

/home/lucas.slattery/clean_repo/system_performance/qcircuitsim/ionq/qcircuitsim/backends/quimb/quimb_mps_results.py:386: ComplexWarning: Casting complex values to real discards the imaginary part
  [max(0, float(d_sv / norm)) for d_sv, norm in zip(discarded_sv_norms, norms, strict=False)]


{'num_coefs_used': 4474,
 'memory_bytes': 71584,
 'estimated_fidelity': (5.750864422247315e-08+0j),
 'fidelity_lower_bound': -13.937708863286264,
 'maybe_discarded_sv_norms': [array(-2.22044605e-16+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(-4.4408921e-16+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(2.22044605e-16+0.j),
  array(5.55111512e-16+0.j),
  array(-1.11022302e-16+0.j),
  array(1.11022302e-16+0.j),
  array(0.+0.j),
  array(0.+0.j),
  array(-3.33066907e-16+0.j),
  array(3.33066907e-16+0.j),
  array(-9.99200722e-16+0.j),
  array(0.+0.j),
  array(4.4408921e-16+0.j),
  array(4.4408921e-16+0.j),
  array(-4.4408921e-16+0.j),
  array(0.+0.j),
  array(2.22044605e-16+0.j),
  array(0.+0.j),
  array(-2.22044605e-16+0.j),
  array(7.77156117e-16+0.j),
  array(-1.11022302e-16+0.j),
  array(5.55111512e-16+0.j),
  arra

In [15]:
result.measurements

{'0': array([0], dtype=uint8),
 '1': array([1], dtype=uint8),
 '2': array([0], dtype=uint8),
 '3': array([1], dtype=uint8),
 '4': array([0], dtype=uint8),
 '5': array([1], dtype=uint8),
 '6': array([0], dtype=uint8),
 '7': array([1], dtype=uint8),
 '8': array([0], dtype=uint8),
 '9': array([0], dtype=uint8),
 '10': array([0], dtype=uint8),
 '11': array([0], dtype=uint8),
 '12': array([0], dtype=uint8),
 '13': array([1], dtype=uint8),
 '14': array([0], dtype=uint8),
 '15': array([0], dtype=uint8),
 '16': array([0], dtype=uint8),
 '17': array([1], dtype=uint8),
 '18': array([0], dtype=uint8),
 '19': array([1], dtype=uint8),
 '20': array([0], dtype=uint8),
 '21': array([1], dtype=uint8),
 '22': array([0], dtype=uint8),
 '23': array([0], dtype=uint8),
 '24': array([0], dtype=uint8),
 '25': array([0], dtype=uint8),
 '26': array([0], dtype=uint8),
 '27': array([0], dtype=uint8),
 '28': array([0], dtype=uint8),
 '29': array([1], dtype=uint8),
 '30': array([0], dtype=uint8),
 '31': array([0], 